In [2]:
import os
import requests
import json
import sys
import os

sys.path.append("/mnt/code")
##Create service accounts and tokens
#nucleus_uri = os.environ['DOMINO_API_HOST']
api_proxy = os.environ['DOMINO_API_PROXY']
svc_accounts_endpoint= f'{api_proxy}/v4/serviceAccounts'


## Why Service Account?

Service Accounts are industry standard ways of creating identities which only interact with the platform programmatically. They are identities used to execute automation workflows.


## What functions does this notebook expose?

1. List all service accounts - `get_all_service_accounts`
2. Create a new service account - `create_sa(sa_name)`
3. Get the Keycloak Id associated with each SA - `get_idp_id_by_sa()`
4. Get the Domino (Mongo) Id associated with each SA - `get_id_by_sa()`
5. Create a named token for a service account. A service account can have many named tokens -`create_token(idpid,token_name)`
6. Invalidate token for a service account. A token that is invalidated cannot be reactivated - `invalidate_token(token_name):`
7. Delete token for a service account. A token with the same name can be recreated after its deleted - `delete_token(token_name)`
8. Add a service account as a project collaborator (collab_id==sa_mongo_id) - `add_sa_as_project_collaborator(project_id,collab_id,project_role)`
9. Get git credential id's associated with all users (id==mongo_id)- `get_git_credentials(id)`
10. Get repo_id associated with a git backed project - `get_project_repo_id(project_id)`
11. Get git credentials associated with a give project - `get_git_credentials`
12. Add a git credential for a collaborating project for a given sa `add_git_cred_for_project_for_sa`
13. Delete git credentials for a collaborating project for a given sa `delete_git_credentials_for_sa`

(11,12,13) in the real world would be run by a SA token after the SA is added as a collaborator for a give project




In [5]:
## First step is fetch all service account
def  get_all_service_accounts():
    print(svc_accounts_endpoint)
    resp = requests.get(svc_accounts_endpoint)
    print(resp)
    users=[]
    if resp.status_code==200:
        for u in resp.json():
            users.append(u['username'])
    return users

def create_sa(sa_name:str):
    data = {'username':sa_name,'email':f'{sa_name}@xyz.com'}
    resp = requests.post(svc_accounts_endpoint,json=data,headers={'Content-Type':'application/json'})
    print(f'Response for creating svc account {sa_name} is {resp.status_code}')
    print(resp.json())

#Now get Idp Id of all emails. This only returns active service accounts. If you have deactivated the SA
#you can no longer see it. The only place to get that is from Mongo or Keycloak to reactivate
#Activation of SA using names is not supported
def get_idp_id_by_sa():
    resp = requests.get(svc_accounts_endpoint)
    print(resp.json())
    idp_by_user={}
    if resp.status_code==200:
        for u in resp.json():
            idp_by_user[u['username']]=u['idpId']
    return idp_by_user
def get_id_by_sa():
    resp = requests.get(svc_accounts_endpoint)
    print(resp.json())
    idp_by_user={}
    if resp.status_code==200:
        for u in resp.json():
            idp_by_user[u['username']]=u['id']
    return idp_by_user
# You cannot overwrite an existing or even an invalidated token. You have to delete it first
def create_token(idpid,token_name):
    token_endpoint= f'{svc_accounts_endpoint}/{idpid}/tokens'    
    resp = requests.post(f"{token_endpoint}",json={"name":token_name},
                         headers={'Content-Type':'application/json'})
    print(resp.status_code)
    return resp.json()

def deactivate_sa(idpid):
    token_endpoint= f'{svc_accounts_endpoint}/{idpid}/deactivate'        
    resp = requests.post(token_endpoint,headers={'Content-Type':'application/json'})
    print(f"Status code {resp.status_code} - Deactivating user {user}")
def activate_sa(idpid):
    token_endpoint= f'{svc_accounts_endpoint}/{idpid}/activate'        
    resp = requests.post(token_endpoint,headers={'Content-Type':'application/json'})
    print(f"Status code {resp.status_code} - Activating user {user}")

#List all tokens
def list_all_tokens(idpid):
    token_endpoint= f'{svc_accounts_endpoint}/{idpid}/tokens'    
    
    resp = requests.get(token_endpoint,headers={'Content-Type':'application/json'})
    
    if resp.status_code==200:
       out = resp.json()           
       pretty_json = json.dumps(out, indent=4)
       print(pretty_json)

def invalidate_token(token_name):
    print(f"Invalidating token {token_name}?")
    token_endpoint= f'{svc_accounts_endpoint}/{idpid}/tokens/{token_name}'    
    resp = requests.post(f"{token_endpoint}/invalidate",headers={'Content-Type':'application/json'})
    print(resp.status_code)
def delete_token(token_name):
    print(f"Deleting token {token_name}?")
    token_endpoint= f'{svc_accounts_endpoint}/{idpid}/tokens/{token_name}'    
    resp = requests.delete(f"{token_endpoint}",headers={'Content-Type':'application/json'})
    print(resp.status_code)
    

In [6]:
print(get_all_service_accounts())

http://localhost:8899/v4/serviceAccounts
<Response [200]>
['svc-user-ds-u1', 'svc-admin-u2', 'svc-user-ds-u3']


In [7]:
## Create Service Accounts if they don't exist
svc_accounts=['svc-user-ds-u1','svc-admin-u2','svc-user-ds-u3']
for sa_name in svc_accounts:
    ## 200 
    create_sa(sa_name)

Response for creating svc account svc-user-ds-u1 is 500
{'message': 'A user with the username svc-user-ds-u1 already exists.', 'success': False}
Response for creating svc account svc-admin-u2 is 500
{'message': 'A user with the username svc-admin-u2 already exists.', 'success': False}
Response for creating svc account svc-user-ds-u3 is 500
{'message': 'A user with the username svc-user-ds-u3 already exists.', 'success': False}


In [8]:
idp_by_user = get_idp_id_by_sa()

[{'username': 'svc-user-ds-u1', 'email': 'svc-user-ds-u1@xyz.com', 'id': '68b9af2aef0f2d1f1e95fdc4', 'idpId': '99d06216-d4bd-444f-9b03-1ed57c28e8ab'}, {'username': 'svc-admin-u2', 'email': 'svc-admin-u2@xyz.com', 'id': '68b9af2b63f3790aeb9ba4b8', 'idpId': '2679d9fa-2efc-4787-a9a5-8800a885de13'}, {'username': 'svc-user-ds-u3', 'email': 'svc-user-ds-u3@xyz.com', 'id': '68b9efbcef0f2d1f1e95fde4', 'idpId': 'd09a9497-0a07-4bb8-8d30-4477c0e6c4e5'}]


In [9]:
## Create tokens for SA. Will return status code 200 if succeeded or 500 if it already exists
results = []
for user, idpid in idp_by_user.items():    
    token_name = f"{user}-token-app"    
    j = create_token(idpid,token_name)
    results.append(j)

with open("/tmp/tokens.json", "w") as file:
    json.dump(results, file, indent=2)

500
500
500


In [10]:
## Activate/Deactivate SVC Tokens. Do not use it currently because a token created for a deactivated/activated user
## does not have `sub` field or claims making it unsuable
'''
for user, idpid in idp_by_user.items():
    deactivate_sa(idpid)
for user, idpid in idp_by_user.items():
    activate_sa(idpid)
'''

'\nfor user, idpid in idp_by_user.items():\n    deactivate_sa(idpid)\nfor user, idpid in idp_by_user.items():\n    activate_sa(idpid)\n'

In [11]:
for user, idpid in idp_by_user.items():
    list_all_tokens(idpid)

[
    {
        "name": "svc-user-ds-u1-token-app",
        "serviceAccountIdpId": "99d06216-d4bd-444f-9b03-1ed57c28e8ab",
        "isValid": true,
        "createdAt": "2025-09-04T19:28:34.012Z",
        "expiresAt": "2026-01-02T19:28:33.999Z"
    }
]
[
    {
        "name": "svc-admin-u2-token-app",
        "serviceAccountIdpId": "2679d9fa-2efc-4787-a9a5-8800a885de13",
        "isValid": true,
        "createdAt": "2025-09-04T19:28:34.403Z",
        "expiresAt": "2026-01-02T19:28:34.391Z"
    }
]
[
    {
        "name": "svc-user-ds-u3-token-app",
        "serviceAccountIdpId": "d09a9497-0a07-4bb8-8d30-4477c0e6c4e5",
        "isValid": true,
        "createdAt": "2025-09-04T20:00:02.251Z",
        "expiresAt": "2026-01-02T20:00:02.241Z"
    }
]


In [12]:
## Invalidate and delete tokens examples
for user, idpid in idp_by_user.items():    
    token_name = f"{user}-token-app"
    # Invalidated tokens cannot be used or be revalidated
    invalidate_token(token_name)

for user, idpid in idp_by_user.items():    
    token_name = f"{user}-token-app"
    # You have to delete a token to recreate it
    delete_token(token_name)

Invalidating token svc-user-ds-u1-token-app?
200
Invalidating token svc-admin-u2-token-app?
200
Invalidating token svc-user-ds-u3-token-app?
200
Deleting token svc-user-ds-u1-token-app?
200
Deleting token svc-admin-u2-token-app?
200
Deleting token svc-user-ds-u3-token-app?
200


In [14]:
#Now recreate them
## Create tokens for SA. Will return status code 200 if succeeded or 500 if it already exists
results = []
for user, idpid in idp_by_user.items():    
    token_name = f"{user}-token-app"    
    j = create_token(idpid,token_name)
    results.append(j)

with open("/tmp/tokens.json", "w") as file:
    json.dump(results, file, indent=2)

201
201
201


In [ ]:

def add_sa_as_project_collaborator(project_id,collab_id,project_role):
    project_endpoint= f'{api_proxy}/v4/projects/{project_id}/collaborators'
    data = {"collaboratorId":collab_id,
            "projectRole":project_role}
    resp = requests.post(url=project_endpoint,json=data,headers={'Content-Type':'application/json'})
    print(resp.status_code)
    print(resp.json())
    

In [ ]:
## As an admin add the SA to the project as a collaborator. This is needed to allow them to run jobs
project_id = os.environ['DOMINO_PROJECT_ID']
id_by_user = get_id_by_sa()
for u,id in id_by_user.items():
    add_sa_as_project_collaborator(project_id,id,"Contributor")


In [ ]:
## Now we need to add a git credential to a service account

In [ ]:
def delete_git_credentials_for_sa(id,credential_id):
    api_proxy = os.environ['DOMINO_API_PROXY']
    headers = {'Content-Type':'application/json'}
    #api_host = os.environ['DOMINO_API_HOST']
    #headers = {'Content-Type':'application/json', 'Authorization': f"Bearer {svc_token}"}
    url = f"{api_proxy}/v4/accounts/{id}/gitcredentials/{credential_id}"    
    resp = requests.delete(url,headers=headers)   
    print(resp.status_code)
    return resp.json()

def add_git_credentials_to_sa(id,git_token_name,git_token):
    #api_host = os.environ['DOMINO_API_HOST']
    #headers = {'Content-Type':'application/json', 'Authorization': f"Bearer {svc_token}"}
    api_proxy = os.environ['DOMINO_API_PROXY']
    headers = {'Content-Type':'application/json'}

    url = f"{api_proxy}/v4/accounts/{id}/gitcredentials"
    p = {"name":git_token_name,"gitServiceProvider":"github","accessType":"token", "token":git_token,"type":"TokenGitCredentialDto"}
    resp = requests.post(url,json=p,headers=headers)   
    print(resp.status_code)
    print( resp.json())
    return resp.json()

def get_git_credentials(id):  
    api_proxy = os.environ['DOMINO_API_PROXY']
    headers = {'Content-Type':'application/json'}
    
#    api_host = os.environ['DOMINO_API_HOST']
#    headers = {'Content-Type':'application/json', 'Authorization': f"Bearer {svc_token}"}

    url = f"{api_proxy}/v4/accounts/{id}/gitcredentials"    
    resp = requests.get(url,headers=headers)   
    print(resp.status_code)
    print(resp.text)
    return resp.json()

In [ ]:
def get_project_repo_id(project_id,svc_token):
    api_host = os.environ['DOMINO_API_HOST']
    headers = {'Content-Type':'application/json', 'Authorization': f"Bearer {svc_token}"}
    
    url = f"{api_proxy}/v4/projects/{project_id}"    
    obj = requests.get(url,headers=headers).json()
    print(obj)
    git_repository_id = obj['mainRepository']['id']
    return git_repository_id

##Run this using the service account token. This is for illustration only
def add_git_cred_for_project_for_sa(project_id,repo_id,cred_id,svc_token):
    api_host = os.environ['DOMINO_API_HOST']
    headers = {'Content-Type':'application/json', 'Authorization': f"Bearer {svc_token}"}

    url = f"{api_host}/v4/projects/{project_id}/repository/{repo_id}/credentialMapping"
    out = requests.put(url,json={"credentialId": cred_id},headers=headers)
    print(out.status_code)
    print(out.text)

def get_git_cred_for_project_for_sa(project_id,repo_id,svc_token):
    api_host = os.environ['DOMINO_API_HOST']
    headers = {'Content-Type':'application/json', 'Authorization': f"Bearer {svc_token}"}
    url = f"{api_proxy}/v4/projects/{project_id}/repository/{repo_id}/credentialMapping"
    out = requests.get(url,headers=headers)
    print(out.status_code)
    print(out.text)

def delete_git_cred_for_project_for_sa(project_id,repo_id,svc_token):
    api_host = os.environ['DOMINO_API_HOST']
    headers = {'Content-Type':'application/json', 'Authorization': f"Bearer {svc_token}"}

    url = f"{api_proxy}/v4/projects/{project_id}/repository/{repo_id}/credentialMapping"
    out = requests.delete(url,headers=headers)
    print(out.status_code)
    print(out.text)


In [ ]:
project_id = os.environ['DOMINO_PROJECT_ID']
id_by_user = get_id_by_sa()
svc_token = os.environ['SVC_TOKEN']
token_name = "my-token"
u = os.environ['SVC_USER']
id = os.environ['SVC_MONGO_ID']

print(f"{u} - {id}")

## Ideally I want to run this as svc account user but i can only run it as admin
creds = get_git_credentials(id)
print(creds)


for c in creds:    
    credential_id = c['id']
    delete_git_credentials_for_sa(id,credential_id)
print(add_git_credentials_to_sa(id,token_name,os.environ['SVC_GITHUB_TOKEN']))
creds = get_git_credentials(id)
print(creds)

In [ ]:

project_id = os.environ['DOMINO_PROJECT_ID']
id_by_user = get_id_by_sa()
repo_id = get_project_repo_id(project_id,svc_token)
print(f"Repo id {repo_id}")

In [ ]:
##This should be run using each service account tokens 
#for u,id in id_by_user.items():
u = "svc-user-ds-u3"  
id = "68b9af2aef0f2d1f1e95fdc4" #os.environ['SVC_MONGO_ID']

creds = get_git_credentials(id)
print(creds)
credential_id = creds[0]['id']
print("Get Git Cred mapping")
print(get_git_cred_for_project_for_sa(project_id,repo_id,svc_token))

print("Delete Git Cred mapping")
delete_git_cred_for_project_for_sa(project_id,repo_id,svc_token)

print("Add Git Cred mapping")
add_git_cred_for_project_for_sa(project_id,repo_id,credential_id,svc_token)

In [ ]:
## And now you can use the service account token to run jobs for a git backed project

In [ ]:
import base64
import json

def decode_jwt(token: str):
    header_b64, payload_b64, signature_b64 = token.split('.')

    # Base64URL decoding requires padding correction
    def b64url_decode(b64_str):
        padding = '=' * (-len(b64_str) % 4)
        return base64.urlsafe_b64decode(b64_str + padding)

    header = json.loads(b64url_decode(header_b64))
    payload = json.loads(b64url_decode(payload_b64))
    
    return {
        "header": header,
        "payload": payload,
        "signature": signature_b64  # raw, usually verified separately
    }

# Example usage
decoded = decode_jwt(svc_token)
print(json.dumps(decoded, indent=2))

In [3]:
from domino import Domino
from utils import domino_utils

svc_token=os.environ['SYS_ACCOUNT_TOKEN')
domino = Domino("wadkars/ddl-end-to-end-demo",auth_token=svc_token)
compute_env_id = domino_utils.get_environment_id('Domino Standard Environment Py3.10 R4.4')['id']
title = f"Test Job for SVC Account"
j = domino.job_start(title=title,command="/tmp/ls",
                 hardware_tier_name="Small",environment_id=compute_env_id)
print(j)

NameError: name 'svc_token' is not defined